In [6]:
import nltk
import glob
import os
import random
import numpy as np 
import pickle
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import tensorflow as tf
from tensorflow import keras

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
DATA_DIR = "/Users/AlexPowers/projects/thee-flying-chicken/ml_scraping/data/"

In [4]:
negMovie = []
posMovie = []
for f_name in glob.glob(os.path.join(DATA_DIR,"movie_reviews","neg","*")):
    negMovie.append(open(f_name, 'r').read())

for f_name in glob.glob(os.path.join(DATA_DIR,"movie_reviews", "pos", "*")):
    posMovie.append(open(f_name,'r').read())

In [4]:
negShort = open(os.path.join(DATA_DIR, "pos_neg_data", "neg.txt"), 'r').read()
posShort = open(os.path.join(DATA_DIR, "pos_neg_data", "pos.txt"), 'r').read()


In [5]:
documents = []
all_words = []
#  j is adjective, r is adverb, and v is verb
allowed_word_types = ["J", "V", "R"]
lemmatizer = WordNetLemmatizer()

for r in negShort.split("\n"):
    documents.append( (r, 1) )
    words = word_tokenize(r)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(lemmatizer.lemmatize( w[0].lower()) )
for r in posShort.split("\n"):
    documents.append( (r, 0) )
    words = word_tokenize(r)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(lemmatizer.lemmatize( w[0].lower()) )


for r in negMovie:
    documents.append( (r, 1) )
    words = word_tokenize(r)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(lemmatizer.lemmatize( w[0].lower()) )

for r in posMovie:
    documents.append( (r, 0) )
    words = word_tokenize(r)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(lemmatizer.lemmatize( w[0].lower()) )

In [8]:
all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())[:len(all_words.keys())//5]

In [9]:
def find_features(document):
    words = word_tokenize(document)
    lem_words = [lemmatizer.lemmatize(w) for w in words]
    features = []
    for w in word_features:
        features.append(int(w in lem_words))

    return features

In [12]:
len(documents)

12664

In [31]:
random.shuffle(documents)
featuresets = [(find_features(rev), category) for (rev, category) in documents[:5000]]

In [32]:
x_train, x_test, y_train, y_test = train_test_split(np.array([i[0] for i in featuresets]), np.array([i[1] for i in featuresets]), test_size=0.2, random_state=42)

In [33]:
x_train.shape

(4000, 5803)

In [34]:
y_train.shape

(4000,)

In [35]:
x_train.min(), x_train.max()

(0, 1)

In [36]:
y_train.min(), y_train.max()

(0, 1)

In [ ]:
clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1, verbose=1)
clf.fit(x_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   22.9s


In [ ]:
print(classification_report(y_test, clf.predict(x_test)))

save_classifier = open(os.path.join(DATA_DIR, "picked_algos", "RandomForestClassifier.pickle"),"wb")
pickle.dump(clf, save_classifier)
save_classifier.close()